In [12]:
# Extraccion de texto de página web
# ==============================================================================
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys

# Librerías de apoyo
# ==============================================================================
import time
import re
from tqdm import tqdm

# Tratamiento de datos
# ==============================================================================
import pandas as pd

# Tratamiento de texto
# ==============================================================================
from nltk.corpus import stopwords
from nltk.tokenize.regexp import regexp_tokenize
import spacy_spanish_lemmatizer
import es_core_news_sm
import string

# Visualización Jupyter
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# otras configuraciones
# ==============================================================================
tqdm.pandas()

In [2]:
%%time
chrome_driver_path = "/usr/lib/chromium-browser/chromedriver"
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)

CPU times: user 5.02 ms, sys: 26 ms, total: 31.1 ms
Wall time: 1.35 s


In [3]:
%%time
driver.get("https://diariooficial.elperuano.pe/Normas")
time.sleep(3)

CPU times: user 1.46 ms, sys: 5.53 ms, total: 6.98 ms
Wall time: 9.53 s


In [4]:
%%time
seach_date_begin = driver.find_element_by_id("cddesde")
seach_date_end = driver.find_element_by_id("cdhasta")
seach_date_begin.clear()
seach_date_end.clear()
seach_date_begin.send_keys("01/07/2021")
seach_date_end.send_keys("16/08/2021")
seach_date_end.send_keys(Keys.RETURN)
seach_date_end.send_keys(Keys.RETURN)
time.sleep(3)

CPU times: user 11.7 ms, sys: 747 µs, total: 12.4 ms
Wall time: 3.28 s


In [5]:
%%time
page_source = driver.page_source
pretty_page = BeautifulSoup(page_source, "lxml")
laws = pretty_page.find_all("article", class_="edicionesoficiales_articulos")
len(laws)

CPU times: user 1.24 s, sys: 51.1 ms, total: 1.29 s
Wall time: 2.78 s


2124

In [6]:
%%time
data = []

for law in tqdm(laws):
    row = {}
    row["category"] = law.select_one(".ediciones_texto h4").text
    row["title"] = law.select_one(".ediciones_texto h5").text
    row["date"] = law.select_one(".ediciones_texto p b").text.replace("Fecha: ", "").strip()
    row["abstract"] = law.select_one(".ediciones_texto p:nth-of-type(2)").text
    content_url = law.select_one(".ediciones_texto h5 a")["href"]
    content_page = requests.get(content_url, verify=False)
    paragraphs = BeautifulSoup(content_page.content).find_all(class_="cuerpo")
    row["content"] = " ".join([paragraph.text for paragraph in paragraphs])
    data.append(row)

100%|██████████| 2124/2124 [18:28<00:00,  1.92it/s]  

CPU times: user 1min 33s, sys: 2.22 s, total: 1min 35s
Wall time: 18min 28s


In [7]:
dataset_base = pd.DataFrame(data)

In [8]:
dataset_base.head()

,category,title,date,abstract,content
0,ECONOMIA Y FINANZAS,RESOLUCION MINISTERIAL N° 250-2021-EF/43,16/08/2021,Designan Asesora de Gabinete de Asesores del D...,"Lima,13 de agosto del 2021 CONSIDERANDO: Que, ..."
1,EDUCACION,RESOLUCION MINISTERIAL N° 301-2021-MINEDU,16/08/2021,Designan Secretaria General del Ministerio,"Lima, 13 de agosto de 2021 VISTOS, el Expedien..."
2,MUJER Y POBLACIONES VULNERABLES,RESOLUCION MINISTERIAL N° 212-2021-MIMP,16/08/2021,Aprueban el Cuadro de Equivalencias de Órganos...,"Lima, 15 de agosto de 2021 Vistos, el Informe ..."
3,MUJER Y POBLACIONES VULNERABLES,RESOLUCION MINISTERIAL N° 214-2021-MIMP,16/08/2021,Designan Secretario General del Ministerio,"Lima, 15 de agosto de 2021 CONSIDERANDO: Que, ..."
4,SUPERINTENDENCIA NACIONAL DE SERVICIOS DE SANE...,RESOLUCION N° 011-2021-SUNASS-DAP,16/08/2021,Determinan el Área de Prestación de Servicios ...,"Lima, 11 de agosto de 2021 VISTOS: El document..."


In [9]:
black_text_list = [
    "EL PRESIDENTE A. I. DEL CONGRESO",
    "EL PRESIDENTE DE LA REPÚBLICA CONSIDERANDO",
    "EL PRESIDENTE DEL CONGRESO",
    "LA PRESIDENTA A. I. DEL CONGRESO",
    "LA PRESIDENTA DEL CONGRESO",
    "DE LA REPÚBLICA",
    "POR CUANTO:",
    "EL CONGRESO DE LA REPÚBLICA;",
    "Ha dado la Ley siguiente:",
    "CONSIDERANDO:"
]

stop_words = stopwords.words("spanish")
# stop_words.extend(["", ""])

lm = es_core_news_sm.load()


def text_clean_for_topic(text: str):

    # Remover lista negras de frases
    for black_text in black_text_list:
        text = text.replace(black_text, "")

    # Convertir a minúscula
    txt = text.lower()

    # Remover signos de puntuación
    txt = re.sub('[%s]' % re.escape(string.punctuation), ' ', txt)

    # Remover números
    txt = re.sub(r'\w*\d+\w*', '', txt)

    # Remover caracteres unicode y números excepto tíldes y ñ
    txt = re.sub("[^a-záéíóúüñ ]+", "", txt)

    # Aplicar Lemmatize
    txt = " ".join([word.lemma_ for word in lm(txt) if word.lemma_ not in stop_words])

    # Remover palabras menos de tres caracteres
    txt = re.sub(r'\b\w{1,3}\b', '', txt)

    # Remover espacios vacios
    txt = txt = re.sub(r"\s+", " ", txt)

    # Remover espacios en inicio y fin
    txt = txt.strip()

    return txt

In [13]:
%%time
dataset_base['content_for_topic'] = dataset_base.content.progress_apply(text_clean_for_topic)

100%|██████████| 2124/2124 [04:42<00:00,  7.53it/s]

CPU times: user 4min 37s, sys: 5.25 s, total: 4min 42s
Wall time: 4min 42s


In [15]:
dataset_base.content_for_topic[0][0:1000]

'lima agosto encontrar vacante plaza correspondiente puesto asesor gabinete asesor despacho ministerial ministerio economía finanza contexto resultar necesario designar persona ejerzar función inherente referido puesto conformidad dispuesto orgánico poder ejecutivo regular participación poder ejecutivo nombramiento designación funcionario público texto integrado actualizado reglamento organización función ministerio economía finanza aprobado mediante resolución ministerial resolver artículo único designar señora aurora tassar lafosse puesto asesora gabinete asesor despacho ministerial ministerio economía finanza regístrese comuníquese publique pedro francke ballvé ministro economía finanza'

In [109]:
stop_words = stopwords.words("spanish")
import string
txt = "#123'漢  abcd no , : h-1231, ññ--ño miltón "
#regexp_tokenize(txt, pattern="\s+", gaps=True)
#txt.encode('ascii', 'ignore').decode()
txt = re.sub('[%s]' % re.escape(string.punctuation), ' ', txt)
#txt = re.sub(r'\w*\d+\w*', '', txt)
txt = re.sub("[^a-záéíóúüñ ]+", "",txt)
#txt = re.sub("[^a-z0-9áéíóúüñ ]+", "",txt)
txt = re.sub(r'\b\w{1,3}\b', '', txt)
#txt = txt = re.sub(r"\s+", " ", txt)
#txt = txt.encode('ascii', 'ignore').decode()
#txt = txt.strip()
#txt = " ".join([word for word in txt.split(' ') if word not in stop_words])
#[word for word in txt.split(' ') if word not in stop_words]
regexp_tokenize(txt, pattern="\s+", gaps=True)
#txt


['abcd', 'miltón']

False

In [87]:
%%time
import spacy_spanish_lemmatizer
import es_core_news_sm
# Change "es" to the Spanish model installed in step 2
nlp = es_core_news_sm.load()
nlp.replace_pipe("lemmatizer", "spanish_lemmatizer")
# for token in nlp("fines"):
#     print(token.text, token.lemma_)

for token in nlp(
    """Con estos fines, la Dirección de Gestión y Control Financiero monitorea
       la posición de capital del Banco y utiliza los mecanismos para hacer un
       
       eficiente manejo del capital."""
):
#print(token.text, token.lemma_)

Con con
estos este
fines fin
, ,
la el
Dirección dirección
de de
Gestión gestión
y y
Control control
Financiero financiero
monitorea monitorea

        
       
la el
posición posición
de de
capital capital
del del
Banco banco
y y
utiliza utilizar
los el
mecanismos mecanismo
para para
hacer hacer
un un

       
        
       
       
eficiente eficiente
manejo manejo
del del
capital capital
. .
CPU times: user 531 ms, sys: 13 ms, total: 544 ms
Wall time: 541 ms


In [ ]:
# Enforcanos a normas del ejecutivo, del gobierno central y del congreso
# Identificar la sobrecarga regulatoria
